In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_together.embeddings import TogetherEmbeddings

from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import uuid

In [ ]:
# MODELS
from models import get_together_fn_mix
# GPT_4_LLM = ChatOpenAI(max_retries=0, model="gpt-4")
ACTIVE_LLM = get_together_fn_mix()

In [ ]:
loaders = [
    TextLoader("sample.txt"),
    # TextLoader("example.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

In [ ]:
docs

In [ ]:
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

In [ ]:

chain = (
    {"doc": lambda x: x.page_content}
    # Only asking for 3 hypothetical questions, but this could be adjusted
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer:\n\n{doc}"
    )
    | ACTIVE_LLM.bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    | JsonOutputParser()
)

In [ ]:
response = chain.invoke(docs[0])

In [ ]:
# Get the questions as an array
questions = response[0]['arguments']['questions']
for q in questions:
    print(q)